In [1]:
from __future__ import print_function
import requests

In [2]:
def get_ans():
	'''Asks user for input. If the user types yes true is returned'''
	ans = input()
	if ans.lower == 'y' or ans.lower == 'yes' or ans == '1':
		return True
	else:
		return False

def get_wcif(comp):
	'''Pulls WCIF from WCA and return json object. If there is a problem with the request the program terminates'''
	print('Attempting to pull competition info...')
	# Send GET request for WCIF
	WCIF = requests.get('https://www.worldcubeassociation.org/api/v0/competitions/' + comp + '/wcif/public')
	# If response code is 200 everything good, otherwise something bad
	if WCIF.status_code == 200:
		print('Success!')
		return WCIF.json()
	else:
		print('Response Error!')
		exit()

In [3]:
# Get comp name from user
print('Enter competition ID:')
#comp = input()
comp = 'Cubinginthe6ix2019'
#comp_file = comp + '-registration.csv'

WCIF = get_wcif(comp)

Enter competition ID:
Attempting to pull competition info...
Success!


In [61]:
class Competitor:
    def __init__(self, name, ID, events, wcaid):
        self.name = name
        self.id = ID
        self.events = events
        self.wcaid = wcaid
        
    @staticmethod
    def build_competitors(WCIF):
        lst = [None] * len(WCIF['persons'])
        i = 0
        for pers in WCIF['persons']:
            lst[i] = Competitor(pers['name'], pers['registrantId'], pers['registration']['eventIds'], pers['wcaId'])
            if lst[i].wcaid == None:
                lst[i].wcaid = ''
            i += 1
        return lst
    
    def write_nametags(comp, f_name, num_blank = 0):
        f = open(f_name, 'w')
        for i in range(len(comp)):
            f.write('\\nametag{%s}{COMPETITOR}{%s}%%\n' % (comp[i].name, comp[i].wcaid))
        for i in range(num_blank):
            f.write('\\nametag{}{COMPETITOR}{}%%\n')
        f.close()

In [62]:
c = Competitor.build_competitors(WCIF)

In [63]:
Competitor.write_nametags(c, 'test.tex')